In [2]:
import os
import sys
import glob
import pandas as pd

In [8]:
id_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/ID_table"
map_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable"
result_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/Mapping_table/QTL"
id_list = glob.glob("%s/*" % id_dir)
modification_list = ["m1A","m5C","m6A","m7G","m6Am","RNA Editing","pseudoU","Nm"]
map_list = [x for x in glob.glob("%s/*.txt" % map_dir) if os.path.basename(x).split(".")[0] not in modification_list]
map_list.remove('/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/RNA_Editing.txt')
print(len(id_list))
print(map_list)

16
['/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/ClinVar.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/pQTL.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/meQTL.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/COSMIC.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/GWAS.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/circQTL.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/eQTL.txt', '/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/bigtable/sQTL.txt']


In [9]:
id_dict, id_col_dict = {}, {}
for id_file in id_list:
#     print(id_file)
    df = pd.read_table(id_file, sep="\t", na_filter = False)
    id_dict[os.path.basename(id_file).split(".")[0]] = dict(zip(df.iloc[0:, 0], df.iloc[0:, 1]))
    id_col_dict[os.path.basename(id_file).split(".")[0]] = df.columns[1]

In [75]:
print(id_dict["circQTL"])

{'circQTL_ID_0001': 'chr10:100913305-100913306|SLF2|chr10:102675756-102685776|rs3802725|cis|4.82E-7|Dorsolateral prefrontal cortex|SCZ or affective/mood disorder|31109370', 'circQTL_ID_0002': 'chr10:100924622-100924623|SLF2|chr10:102675756-102685776|rs10883563|cis|2.03E-10|Dorsolateral prefrontal cortex|SCZ or affective/mood disorder|31109370', 'circQTL_ID_0003': 'chr10:100986473-100986474|SLF2|chr10:102675756-102685776|rs7184|cis|0.00000287|Dorsolateral prefrontal cortex|SCZ or affective/mood disorder|31109370', 'circQTL_ID_0004': 'chr10:102813205-102813206|AS3MT|chr10:104632205-104634957|rs284860|cis|2.67E-7|Dorsolateral prefrontal cortex|SCZ or affective/mood disorder|31109370', 'circQTL_ID_0005': 'chr10:102813205-102813206|BORCS7-ASMT|chr10:104632205-104634957|rs284860|cis|2.67E-7|Dorsolateral prefrontal cortex|SCZ or affective/mood disorder|31109370', 'circQTL_ID_0006': 'chr10:102813645-102813646|AS3MT|chr10:104632205-104634957|rs11191401|cis|5.82E-8|Dorsolateral prefrontal cortex

In [10]:
print(id_col_dict["circQTL"])

circQTL_QTL SNP Position|circQTL_Gene Name|circQTL_RS ID|circQTL_QTL Type|circQTL_Pvalue|circQTL_Tissue/Cell|circQTL_Condition|circQTL_PubMed ID|circQTL_circRNA Position


In [11]:
def replace_content(in_list, map_dict):
    reversed_map = {v: k for k, v in map_dict.items()}
    result_list = []
    for line in in_list:
        new_line = "!!!!!!!!!!!!!!!!!!!!!!!"
        if line == "NA":
            new_line = "NA"
        else:
            separator = "@"
            if "#" in line:
                separator = "?"
#                 print(line)
#                 sys.exit(1)
            term_list, new_list = line.split(separator), []
            for term in term_list:
                if term == "NA":
                    new_list.append("NA")
                elif term in reversed_map:
                    new_list.append(reversed_map[term])
#                 else:
#                     print(term)
#                     print(line)
#                     sys.exit(1)
            new_line = separator.join(new_list)
        result_list.append(new_line)
    return result_list

In [78]:
def reformat_modificiation(in_list):
    out_list= []
    for line in in_list:
        if line == "NA":
            out_list.append(line)
        else:
            term_list, new_list = line.split("?"), []
            for term in term_list:
                if len(term.split("#")) < 4:
                    new_list.append(term)
                else:
                    prefix_term = "#".join([term.split("#")[0], term.split("#")[2], term.split("#")[3]])
                    post_term = term.split("#")[4]
                    new_line = prefix_term + "#" + post_term
                    new_list.append(new_line)
            out_list.append("?".join(new_list))
    return out_list

In [80]:
for map_file in map_list:
    print(os.path.basename(map_file))
    df_map = pd.read_table(map_file, sep="\t", na_filter = False)
    # process first column
    query_name = os.path.basename(map_file).split(".txt")[0]
    first_values = replace_content(df_map.iloc[:, 0], id_dict[query_name])
    df_map.iloc[:, 0] = first_values
    # process remaing columns
    remaining_cols = [x for x in df_map.columns[2:] if "splicing" not in x]
    for col in remaining_cols:
#         print(col)
        if col.startswith("m6A") or col.startswith("PseudoU") or col.startswith("m5C"):
            new_contents = replace_content(reformat_modificiation(df_map[col]), id_dict[col.split("_")[0]])
#             df_map = df_map.rename(columns={col: id_col_dict[col.split("_")[0]]})
        else:
            new_contents = replace_content(df_map[col], id_dict[col.split("_")[0]])
        df_map[col] = new_contents
    # print(df_map.head())
#     remaining_cols = [x for x in df_map.columns[2:] if "splicing" not in x]
    df_final = df_map[[df_map.columns[0]] + remaining_cols]
    result_file = os.path.join(result_dir, os.path.basename(map_file))
    df_final.to_csv(result_file, sep="\t", index=False)
#     break

pQTL.txt
meQTL.txt
clinvar.txt
COSMIC.txt
GWAS.txt
circQTL.txt
eQTL.txt
sQTL.txt


In [85]:
# assign number by xiao cui
id_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/ID_table"
gwas = os.path.join(id_dir, "GWAS_Name.txt")
clinvar = os.path.join(id_dir, "clinvar_Name.txt")
cosmic = os.path.join(id_dir, "COSMIC_Name.txt")
# 
map_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/Mapping_table/QTL"
gwas_map = os.path.join(map_dir, "GWAS.txt")
clinvar_map = os.path.join(map_dir, "clinvar.txt")
cosmic_map = os.path.join(map_dir, "COSMIC.txt")

result_dir = "/home/galaxy/project/m6AQTL/data/data_for_website/final_data/ID_index/Mapping_table/QTL/with_disease_name"
def insert_column(file1, file2, outfile):
    df_1 = pd.read_table(file1, sep="\t")
    id_list = df_1.iloc[:, 0].tolist()
    df_2 = pd.read_table(file2, sep="\t")
    df_2.insert(0, df_1.columns[0], id_list)
    df_2.to_csv(outfile, sep="\t", index=False)

insert_column(gwas, gwas_map, os.path.join(result_dir, "GWAS.txt"))
insert_column(clinvar, clinvar_map, os.path.join(result_dir, "clinvar.txt"))
insert_column(cosmic, cosmic_map, os.path.join(result_dir, "COSMIC.txt"))